In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

## Store Part I results into DataFrame
* Load the CSV exported in Part I to a DataFrame

In [7]:
city_data_df = pd.read_csv("output_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,makhachkala,42.98,47.50,41.00,93,90,4.47,RU,1609365944
1,1,jiblah,13.92,44.15,57.69,85,23,4.65,YE,1609366061
2,2,rikitea,-23.12,-134.97,78.30,76,0,11.92,PF,1609366061
3,3,qaanaaq,77.48,-69.36,8.08,73,67,13.29,GL,1609366061
4,4,hermanus,-34.42,19.23,64.00,80,4,10.71,ZA,1609366061


## Humidity Heatmap
* Configure gmaps
* Use the Lat and Lng locations and Humidity as the weight.
* Add Heatmap layer to map

In [8]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [10]:
# Heatmap of humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

## Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions
* Drop any rows will null values

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
21,21,busselton,-33.65,115.33,72.00,52,0,3.00,AU,1609365808
47,47,caravelas,-17.71,-39.25,76.39,81,0,7.65,BR,1609366077
103,103,taoudenni,22.68,-3.98,73.33,24,0,7.29,ML,1609366093
106,106,kaya,13.09,-1.08,72.88,18,0,8.52,BF,1609366094
111,111,ler,8.30,30.14,77.76,30,0,4.47,SS,1609366096
131,131,el castillo,19.53,-96.85,71.28,55,0,4.92,MX,1609366062
148,148,nioro,13.35,-15.75,75.83,45,0,5.91,GM,1609366108
150,150,umm lajj,25.02,37.27,73.36,69,0,8.41,SA,1609366109
171,171,jizan,17.33,42.67,78.80,78,0,8.05,SA,1609365963
189,189,sambava,-14.27,50.17,74.89,85,0,9.62,MG,1609366119


## Hotel Map
* Store into variable named hotel_df.
* Add a "Hotel Nanme" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap

In [14]:
# Create DataFrame called hotel_df to store hote names along with city, country and coordinates
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
21,busselton,AU,-33.65,115.33,
47,caravelas,BR,-17.71,-39.25,
103,taoudenni,ML,22.68,-3.98,
106,kaya,BF,13.09,-1.08,
111,ler,SS,8.30,30.14,
131,el castillo,MX,19.53,-96.85,
148,nioro,GM,13.35,-15.75,
150,umm lajj,SA,25.02,37.27,
171,jizan,SA,17.33,42.67,
189,sambava,MG,-14.27,50.17,


In [16]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
21,busselton,AU,-33.65,115.33,Observatory Guest House
47,caravelas,BR,-17.71,-39.25,Pousada dos Navegantes
103,taoudenni,ML,22.68,-3.98,
106,kaya,BF,13.09,-1.08,Maison de l'appelé
111,ler,SS,8.30,30.14,
131,el castillo,MX,19.53,-96.85,City Express Xalapa
148,nioro,GM,13.35,-15.75,
150,umm lajj,SA,25.02,37.27,Moon light Furnished Units
171,jizan,SA,17.33,42.67,شهدان. ابوالحجر
189,sambava,MG,-14.27,50.17,Melrose


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat","Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure 
fig

Figure(layout=FigureLayout(height='420px'))